In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
!pwd
import sys
import os
sys.path.append(os.path.abspath('../'))

/Users/rabauti/repos/eki/prg1978/heete_loendid


In [8]:
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.tasak_reader import TasakReader

# functions for creating database and collecting collocations
from collect_functions_2_1 import *

# Tabel 2.1 andmete kogumine


"Järgmiseks võiks vaadata selliseid, kus da-infi märgend on csubj ja root verb on 3sg (ainsuse kolmandas isikus). Ühes veerus võiks ära märkida, kas root verbil on allatiivis või adessiivis alluv obl. St välja võiks tulla laused nagu Mulle meeldib lugeda, Mul õnnestus lugeda."

Tasakaalus korpusest kogutakse kokku järgmisele mallile vastavad esinemised:

* peaverb on ainsuse 3 pöördes ps3;
* peaverbi alluvate seas on csub
* csubj on da-infinitiivis (kategooriare seas on "inf")



Skripti töö tulemusena tekitatakse kaks tabelit:

### Põhitabel
|veerg|	kirjeldus| filter |
|--|--|--|
|verb|	verbi lemma| verbil pole atribuute "imper" või "imps", verbi deprel pole "aux"; verb on ps3|
|csubj|	verbi alluv csubj lemma, filtreeritud: inf| |
|has_obl_ad_all| 	verbil esines obl alluv adessiivis või allatiivis (ad, all) | |
|total|	kokku absoluutseid esinemisi Tasakaalus korpuses| |
	
	
	
### Näited	
|veerg|	kirjeldus|
|--|--|
|sentence_id|	lause id korpuses (hetkel pole sellest suurt kasu)|
|verb	|peaverbi lemma|
|sentence_part|lause osa, kus on verb+vahetud verbi alluvad lauses esinemise järjekorras|
|sentence_text	|lause kogu tekst|
|verb_id	|tegusõna sõna number lauses (hetkel pole sellest suurt kasu)|


In [9]:
%%time

file_name = '../data/tasak.vert'
my_reader = TasakReader(
   file_name = file_name
)

CPU times: user 14 µs, sys: 17 µs, total: 31 µs
Wall time: 31.9 µs


In [10]:
%%time

TYPE = 'verb_ps3_csubj_vda'
TABLENAME = f'{TYPE}'
BATCH_SIZE = 100000

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"tasak_test_{TYPE}_collocations_{date_time}.db"

my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


# hoitakse algusest lõpuni mälus
examples = {}

# kollokatsioonid, tühjendatakse peale igat salvestamist
collocations = {}
count = 0
for collection_id, graph in my_reader.get_sentences():
    
    count += 1
    if not collection_id:
        collection_id = count

    #print(f'---------- {count} --- {collection_id} -----------')
    
    #print(str(graph))
    #for n in sorted(graph.nodes): print(graph.nodes[n])
    #print('')
    #print(' '.join([graph.nodes[n]['form'] for n in sorted(graph.nodes) if n]))

    #if count <= 100: graph.draw_graph()

    
    collocations, examples, = extract_something(graph, collection_id, collocations, examples )
    

    #for c in collocations:
    #    print(c, collocations[c])
    #print()

    if not collection_id == 0 and not count % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = {}
    

    
    
# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)
collocations = {}
my_sqlite_db.index_fields()


../data/tasak.vert


TSV lines:   9%|▉         | 1803434/20058039 [00:06<01:10, 258327.77it/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest:0 - 346546


TSV lines:  18%|█▊        | 3582899/20058039 [00:13<01:02, 264351.95it/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest:0 - 4759049


TSV lines:  26%|██▌       | 5241007/20058039 [00:19<00:55, 265788.90it/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest:0 - 7050913


TSV lines:  28%|██▊       | 5703028/20058039 [00:21<00:53, 270506.07it/s]